# Michigan Covid Scraper - using BeautifulSoup
Scrape COVID-19 case counts from michigan.gov website.

In [1]:
import requests
import pandas as pd
import bs4

In [2]:
url='https://www.michigan.gov/coronavirus/0,9753,7-406-98163-520743--,00.html'

Get the text of the page using requests.

In [3]:
page_text = requests.get(url).text
# page_text

Use BeautifulSoup to parse text and turn into soup object.

In [4]:
page_soup = bs4.BeautifulSoup(page_text)

Since looks like only one table on page, will just find first table.

In [5]:
stats_table = page_soup.table

Check if we got we think. Yep.

In [6]:
stats_table.contents

['\n',
 <caption><strong>Overall Confirmed COVID-19 Cases by County</strong></caption>,
 '\n',
 <thead>
 </thead>,
 '\n',
 <tbody>
 <tr>
 <td nowrap="nowrap" style="width: 251px;"><strong>  County</strong></td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"><strong>Cases</strong></td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"><strong>Deaths</strong></td>
 </tr>
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Allegan</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">1</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"> </td>
 </tr>
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Barry</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">1</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"> </td>
 </tr>
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Bay</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">3</td>
 <td nowrap="nowrap" style="width: 88px; text-align: cen

Get all the rows of the table. The result is a list.

In [7]:
stats_table_rows = stats_table.find_all('tr')
stats_table_rows[:5]

[<tr>
 <td nowrap="nowrap" style="width: 251px;"><strong>  County</strong></td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"><strong>Cases</strong></td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"><strong>Deaths</strong></td>
 </tr>,
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Allegan</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">1</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"> </td>
 </tr>,
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Barry</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">1</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"> </td>
 </tr>,
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Bay</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;">3</td>
 <td nowrap="nowrap" style="width: 88px; text-align: center;"> </td>
 </tr>,
 <tr>
 <td nowrap="nowrap" style="width: 251px;">  Berrien</td>
 <td nowrap="nowrap" style="width: 88px; te

Now we can just iterate over the list (the rows). For each row:

* create empty master list
* find all the table data elements
* use a list comprehension to create a list of elements in each row. Note use of `.strip()` to clean county name.
* do string to number conversions
* as long as resulting list isn't None, append to our master list

In [8]:
stats_data_rows = []
for tr in stats_table_rows:
    tds = tr.find_all('td')
    row = [td.text.strip().replace(",","") for td in tds]
    # Let's do the string to int conversions now (but skip the header)
    if len(stats_data_rows) > 0:
        for i in range(1, 3):
            if row[i] == '':
                row[i] = 0
            else:
                row[i] = int(row[i])
    if row:
        stats_data_rows.append(row)

In [9]:
stats_data_rows

[['County', 'Cases', 'Deaths'],
 ['Allegan', 1, 0],
 ['Barry', 1, 0],
 ['Bay', 3, 0],
 ['Berrien', 10, 0],
 ['Calhoun', 6, 0],
 ['Charlevoix', 4, 0],
 ['Chippewa', 1, 0],
 ['Clare', 1, 0],
 ['Clinton', 6, 0],
 ['Detroit City', 705, 12],
 ['Eaton', 3, 0],
 ['Emmet', 2, 0],
 ['Genesee', 46, 0],
 ['Gladwin', 2, 0],
 ['Grand Traverse', 3, 0],
 ['Hillsdale', 2, 0],
 ['Ingham', 18, 0],
 ['Iosco', 1, 0],
 ['Isabella', 2, 0],
 ['Jackson', 12, 0],
 ['Kalamazoo', 5, 0],
 ['Kalkaska', 2, 0],
 ['Kent', 36, 1],
 ['Lapeer', 1, 0],
 ['Leelanau', 1, 0],
 ['Lenawee', 1, 0],
 ['Livingston', 16, 1],
 ['Luce', 1, 0],
 ['Macomb', 281, 7],
 ['Manistee', 1, 0],
 ['Marquette', 1, 0],
 ['Midland', 6, 0],
 ['Monroe', 18, 0],
 ['Montcalm', 2, 0],
 ['Muskegon', 3, 0],
 ['Newaygo', 1, 0],
 ['Oakland', 543, 10],
 ['Otsego', 6, 0],
 ['Ottawa', 16, 0],
 ['Roscommon', 1, 0],
 ['Saginaw', 9, 0],
 ['Sanilac', 1, 0],
 ['St. Clair', 10, 0],
 ['Tuscola', 2, 0],
 ['Van Buren', 2, 0],
 ['Washtenaw', 72, 3],
 ['Wayne', 417, 9

In [10]:
stats = pd.DataFrame(stats_data_rows[1:], columns = stats_data_rows[0])
stats.head(50)

,County,Cases,Deaths
0,Allegan,1,0
1,Barry,1,0
2,Bay,3,0
3,Berrien,10,0
4,Calhoun,6,0
5,Charlevoix,4,0
6,Chippewa,1,0
7,Clare,1,0
8,Clinton,6,0
9,Detroit City,705,12


Add the SE Michigan county indicators. Avoid spaces in column names.

In [11]:
stats['InMarket'] = 0
market_counties = ['Detroit City','Lapeer','Macomb','Monroe','Oakland','Genesee','Wayne','St. Clair']
stats.loc[stats['County'].isin(market_counties),['InMarket']] = 1

In [12]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   County    50 non-null     object
 1   Cases     50 non-null     int64 
 2   Deaths    50 non-null     int64 
 3   InMarket  50 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB


The warning you are getting in your version of the following statement is due to deprecation of `[col1, col2]` on `GroupBy` objects. Needs to be a list of columns inside the indexer brackets: `[[col1, col2]]`.

In [13]:
stats.groupby(['InMarket'])[['Cases','Deaths']].sum()

,Cases,Deaths
InMarket,,
0,2565,48
1,2021,38
